In [1]:
# from tqdm import tqdm
from tqdm.notebook import tqdm
from datasets import load_dataset, DatasetDict, load_from_disk
# from transformers import GPT2Tokenizer, GPT2Config, GPT2LMHeadModel, GPT2LMHeadModel
# from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
import accelerate
import transformers
# transformers.__version__, accelerate.__version__
import torch
torch.cuda.is_available()
# from scripts.utils import *

import sys
import os

scripts_dir = os.path.abspath(os.path.join(os.getcwd(), '..', 'scripts'))

if scripts_dir not in sys.path:
    sys.path.append(scripts_dir)

from utils import *

device = "cuda" if torch.cuda.is_available() else "cpu"
print(torch.cuda.get_device_name(0))


ModuleNotFoundError: No module named 'scripts'

In [ ]:
# from argparse import ArgumentParser
# parser = ArgumentParser()
# # parser.add_argument("--input_data_dir")
# # parser.add_argument("--need_to_tokenize")
# parser.add_argument("--tokenized_data_dir")
# parser.add_argument("--model_dir")
# parser.add_argument("--eval", default=False)
# parser.add_argument("--from_pretrained", default=False)
# parser.add_argument("--context_size", default=None)
# parser.add_argument("--context_direction", default='left')

from_pretrained=False
tokenized_data_dir = 'data/coca_spoken/tokens_sentence'
model_dir= 'models/script1/left_sentence'
context_size = None
context_direction = 'left'
evall=False
# args = parser.parse_args()

# Init/load model
if from_pretrained:
    model = load_pretrained_model(model_dir)
else:
    model = init_model(context_size)

model.to(device)
if evall:
    model.eval()

# Load tokenizer
tokenizer = load_pretrained_tokenizer('gpt2', context=context_size)

# Create data collator
data_collator = init_data_collator(tokenizer, context_direction)

train_args = TrainingArguments(
    model_dir,
    per_device_train_batch_size=1024, # change to fit GPU specs
    per_device_eval_batch_size=1024,
    # auto_find_batch_size=True,
    evaluation_strategy='epoch',
    # eval_steps=1,
    # logging_steps=0.01,
    save_strategy='epoch',
    save_steps=0.25,
    group_by_length=True, # bucketing
    # load_best_model_at_end=True,
    # metric_for_best_model='loss',
    greater_is_better=False,
    save_total_limit=5,
    num_train_epochs=1,
)
print(train_args.device)

tokenized_dataset_dict = load_from_disk(tokenized_data_dir)
tokenized_dataset_dict = tokenized_dataset_dict.remove_columns(['text'])

if not evall:
    trainer = Trainer(
        model=model,
        tokenizer=tokenizer,
        args=train_args,
        data_collator=data_collator,
        train_dataset=tokenized_dataset_dict['train'],
        eval_dataset=tokenized_dataset_dict['val'],
    )

    if not from_pretrained:
        trainer.train()
    else:
        trainer.train(
            resume_from_checkpoint=model_dir
        )